In [1]:
import os
from datetime import datetime, timedelta
import geopandas as gpd
import shapely
import sys
import requests
import pandas as pd

from config import Config

In [2]:
config = Config()

In [3]:
def parse_shp(filename, fid):
    """
    Parses a shapefile of fire events to extract specific fire event information.
    The shapefile is the output of FiredPy package.
    """
    
    fires = gpd.read_file(filename)
    if not str(fid) in fires['id'].values:
         print("invalid fire ID provided! Make sure the ID exists in the fire events shapefile")
         sys.exit()
    fire_event = fires[fires['id'] == str(fid)]
    start_date = datetime.strptime(fire_event['ig_date'].values[0], "%Y-%m-%d")
    end_date = datetime.strptime(fire_event['last_date'].values[0], "%Y-%m-%d")
    footprint = fire_event['geometry'].values[0]

    return fid, start_date, end_date, footprint

In [4]:
file_dir = os.path.join(config.data_dir, "Fire_events")
file_name = os.path.join(file_dir, "selected_events.shp")
# file_dir + "selected_events.shp"

fid, start_date, end_date, footprint = parse_shp(file_name, '8676')

print(f"\nSearching for Sentinel-2 images for event ID {fid} between {start_date} and {end_date}")
print(f"\nDownloading images is set to: {config.download_scenes}")

polygon = shapely.geometry.box(*footprint.bounds, ccw=True)
# Search parameters
time_delta = config.delta_days_sentinel
adjusted_start_date = start_date - timedelta(days=time_delta) 
adjusted_end_date = end_date + timedelta(days=time_delta)
adjusted_start_date = adjusted_start_date.isoformat() + 'Z'
adjusted_end_date = adjusted_end_date.isoformat() + 'Z'
product_type = config.producttype_sentinel
max_cloud_cover = config.max_cloud_cover


Searching for Sentinel-2 images for event ID 8676 between 2021-02-21 00:00:00 and 2021-03-29 00:00:00



In [5]:
end_date = end_date.isoformat() + 'Z'

In [6]:
adjusted_start_date

'2021-01-12T00:00:00Z'

In [7]:
adjusted_end_date

'2021-05-08T00:00:00Z'

In [8]:
catalogue_odata_url = "https://catalogue.dataspace.copernicus.eu/odata/v1"
search_period_start = adjusted_start_date
search_period_end = adjusted_end_date
collection_name = "SENTINEL-2"
aoi = polygon.wkt
product_type = config.producttype_sentinel
max_cloud_cover = config.max_cloud_cover

In [14]:
search_query = (f"{catalogue_odata_url}/Products?$filter=Collection/Name eq '{collection_name}' and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/OData.CSC.StringAttribute/Value eq '{product_type}') and OData.CSC.Intersects(area=geography'SRID=4326;{aoi}') and Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value le {max_cloud_cover}) and ContentDate/Start ge {search_period_start} and ContentDate/End le {search_period_end}&$top=50")

In [15]:
json = requests.get(search_query).json()
result = pd.DataFrame.from_dict(json['value'])

In [16]:
len(result)

32

In [17]:
result = result[result['ContentLength'] > 0].reset_index(drop=True)

In [18]:
len(result)

17

In [47]:
result

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,a8f1fed0-f650-455b-819a-aa91955b5fb0,S2A_MSIL2A_20210320T171011_N9999_R112_T14SPC_2...,application/octet-stream,1300054510,2023-03-31T15:02:04.359Z,2023-03-31T17:52:48.034Z,2023-03-31T17:52:48.034Z,True,,/eodata/Sentinel-2/MSI/L2A/2021/03/20/S2A_MSIL...,[{}],"{'Start': '2021-03-20T17:10:11.024Z', 'End': '...",geography'SRID=4326;POLYGON ((-97.91284 34.336...,"{'type': 'Polygon', 'coordinates': [[[-97.9128..."
1,application/octet-stream,7dff1046-234d-45e7-a0a4-bff7e8aeef55,S2A_MSIL2A_20210320T171011_N0500_R112_T14SPC_2...,application/octet-stream,1177552696,2023-06-23T00:23:49.474Z,2023-06-23T00:40:03.911Z,2023-06-23T04:48:56.671Z,True,,/eodata/Sentinel-2/MSI/L2A_N0500/2021/03/20/S2...,"[{'Value': 'eb2f65fb12eb59f3bd9353c9d074f802',...","{'Start': '2021-03-20T17:10:11.024Z', 'End': '...",geography'SRID=4326;POLYGON ((-97.91284 34.336...,"{'type': 'Polygon', 'coordinates': [[[-97.9128..."
2,application/octet-stream,718893c4-62a5-4615-afda-0b5a3a2ec024,S2B_MSIL2A_20210315T170949_N0500_R112_T14SPC_2...,application/octet-stream,1158276063,2023-07-03T11:10:59.075Z,2023-07-03T11:27:59.227Z,2023-07-03T11:28:12.858Z,True,,/eodata/Sentinel-2/MSI/L2A_N0500/2021/03/15/S2...,"[{'Value': '840663c4fc64322022bbb88cedae90cf',...","{'Start': '2021-03-15T17:09:49.024Z', 'End': '...",geography'SRID=4326;POLYGON ((-97.91284 34.336...,"{'type': 'Polygon', 'coordinates': [[[-97.9128..."
3,application/octet-stream,5dda1d6c-aff7-40fd-866d-c2a95d81bec0,S2B_MSIL2A_20210223T171309_N0500_R112_T14SPC_2...,application/octet-stream,1182606920,2023-07-08T19:25:44.320Z,2023-07-08T21:24:05.312Z,2023-07-08T21:24:21.513Z,True,,/eodata/Sentinel-2/MSI/L2A_N0500/2021/02/23/S2...,"[{'Value': '479a2af6eb482f07a03cb97f99927aab',...","{'Start': '2021-02-23T17:13:09.024Z', 'End': '...",geography'SRID=4326;POLYGON ((-97.912851506605...,"{'type': 'Polygon', 'coordinates': [[[-97.9128..."
4,application/octet-stream,49b740b9-46c4-4eec-a97c-0b87034f7e0d,S2B_MSIL2A_20210203T171519_N0500_R112_T14SPD_2...,application/octet-stream,1032755375,2023-06-28T00:53:16.555Z,2023-06-28T01:08:04.432Z,2023-06-28T01:08:18.010Z,True,,/eodata/Sentinel-2/MSI/L2A_N0500/2021/02/03/S2...,"[{'Value': '143e4410ddea6f2613512869b6f49e2d',...","{'Start': '2021-02-03T17:15:19.024Z', 'End': '...",geography'SRID=4326;POLYGON ((-97.90161 35.187...,"{'type': 'Polygon', 'coordinates': [[[-97.9016..."
5,application/octet-stream,d5cb7ddf-789a-42ed-a46a-528af8d6c58c,S2B_MSIL2A_20210223T171309_N0500_R112_T14SPD_2...,application/octet-stream,1196209534,2023-07-08T19:13:06.392Z,2023-07-08T20:06:04.288Z,2023-07-08T20:06:21.950Z,True,,/eodata/Sentinel-2/MSI/L2A_N0500/2021/02/23/S2...,"[{'Value': 'd78ea4eb0ce686362389b0bc30081168',...","{'Start': '2021-02-23T17:13:09.024Z', 'End': '...",geography'SRID=4326;POLYGON ((-97.900959187625...,"{'type': 'Polygon', 'coordinates': [[[-97.9009..."
6,application/octet-stream,12bced87-ec08-443c-b24b-0dc43f611b7d,S2B_MSIL2A_20210213T171419_N0500_R112_T14SPD_2...,application/octet-stream,1177858740,2023-07-03T06:43:38.899Z,2023-07-03T07:14:18.685Z,2023-07-03T07:14:36.244Z,True,,/eodata/Sentinel-2/MSI/L2A_N0500/2021/02/13/S2...,"[{'Value': '15b25febd8d4215824ebf95c94947a4a',...","{'Start': '2021-02-13T17:14:19.024Z', 'End': '...",geography'SRID=4326;POLYGON ((-97.90097 35.238...,"{'type': 'Polygon', 'coordinates': [[[-97.9009..."
7,application/octet-stream,8ae8b400-fd1f-46e6-acb5-593cb258aa25,S2A_MSIL2A_20210218T171341_N9999_R112_T14SPD_2...,application/octet-stream,1381630394,2023-03-22T10:09:41.820Z,2023-03-22T12:06:18.235Z,2023-03-22T12:06:18.235Z,True,,/eodata/Sentinel-2/MSI/L2A/2021/02/18/S2A_MSIL...,[{}],"{'Start': '2021-02-18T17:13:41.024Z', 'End': '...",geography'SRID=4326;POLYGON ((-97.90097 35.238...,"{'type': 'Polygon', 'coordinates': [[[-97.9009..."
8,application/octet-stre